Trending Topics, Last Writeup
------------------------------

For the last two weeks, we have been considering what it means for a topic to be "trending." In principle, lists of trending topics are a way to synthesize the activity in some large information space. But how are they computed? Do they really reflect the actions that people are taking on Google or Instagram or Twitter or Facebook? In what sense to they reflect larger local or global community concerns?

In the end we have a few data sets. (These are the final versions.)

1. Top trending topics on Twitter, sampled every 15 minutes from 64 cities and the US. We focused on the days around the Inauguration, including the Women's March on that Saturday. [Download it here.](http://compute-cuj.org/twitter_trending_topics_for_us_120to122_mh2.csv.gz)
2. A sample of tweets collected from the DC area, around the same time period. Twitter's API provides a (statistical) sample of tweets, although the exact percentage they supply is a bit unclear. This can be used to compare tweets to the trending topics above. [Download it here.](dc_tweets_0120_to_0123_v2_mh2.csv.gz)
3. Last time we provided two derived data sets, one summarizing hashtag usage in the DC tweets data set, and one summarizing user patterns in the same data. [Download the hashtag summary](http://compute-cuj.org/hashtagsummary.csv) or [download the user summary](http://compute-cuj.org/usersummary.csv).

In each case, you download the files, uncompress them (if needed) and place them in the same folder as this notebook. There were other handy data sets, like the big dataset of all hashtags used in tweets from DC and their time. You can use the data sets above and the code from previous notebooks to create those again, if you found them useful.

With these data, we have examined how Twitter's algorithm is working. We explored what trended when, during a weekend that had plenty of events. When did the Inauguration and its various participants trend? Did they? What about the march? When did that start trending and where?

We then examined the anatomy of a tweet -- how  are tweets represented, what data do they contain and how we might collect them from Twitter. We derived various summaries that took the raw tweets (in JSON) and produced simple CSVs to process. We could first just look at how timing, tweet volume, tweet text and user participation all varied. We could also examine hashtag and URL usage as well as other media. Oh and mentions, we could explore who mentions whom, finding conversations in our tweets data set.

Finally, we developed our own metrics for what might be trending. Obviously creating an industrial-strength trending algorithm is hard, and the subject of considerable effort on the part of companies like Twitter and Facebook, but you can get a sense of the complexities by trying out simple rules. How easy is it to "game" the rankings? How does changing the rules change your perception of what's going on? 

In this final drill on trends (although the ideas will come back repeatedly), I'd like you to summarize what we've done. This can come in a couple of forms. 
1. Try out a ranking algorithm, apply it to several time periods, compare it to the underlying tweets and assess how well it did. You might want to start, as we did in the previous drill, by outlining your goals for the recipe you created and how it represents the idea of a "trend". Write it up and explain what  you learned from the experience about trending. 
2. Use a mixture of the data sets above to provide a story about trends during the weekend. In DC, around the country, maybe nationally. This will be more of a data story -- we have collected a rich set of tweets and trends for you. What do they say about that weekend? You don't have enough data to pull off an article like this (yet), but here's [a Buzzfeed piece on tweets during the inauguration.](https://www.buzzfeed.com/gilgul/8-things-we-learned-from-10-million-inauguration-emoji) 

Use this notebook to write your short story. Actually, a story pitch is fine, complete with where you might go to finish it -- who you might talk to, what other data you might need. Feel free to consult as many sources (web, human) as you would like to fill out your understanding of trending beyond what we have provided. I'm looking for something short, maybe 500-800 words. Include your code in this notebook as well. Please use as many cells and document it as best you can.

**Work in your groups, but you should each turn in your own story and your own version of the code.** These can be writeups of the same experiment (and quite similar in content), but I want to have each of you write things up in your own words. That helps us see that you uderstand what the computations are doing. OK?

**Take your time with this.** We want to see that you can both engage with the topic and show that you are able to express your ideas in code. Contact me if you have any problems.

Put your story here, indicating who you worked with.



In [ ]:
# Put your code here, adding as many cells as you need




**Appendix: Functions, revisited**

In the last drill, we created two functions. You learned how to group lines of code into a funtion body and then pass arguments to perform a computation. I was a little sloppy with this first introduction to a function. So let's do it again,  slowly. 

Here is a computation we might want to perform. Given a string representing a sentence, break it into words. We start with the string, turn it lowercase and then split by spaces. (It's a line from Trump's inauguration speech.)

In [ ]:
speech = "The oath of office I take today is an oath of allegiance to all Americans."
speech.lower().split(" ")

Let's wrap this into a function. We will call the argument **sentence** (because we might want to apply it to something other than the inauguration speech). We then take the sentence and carve it up into words. 

In [ ]:
def wordfun(sentence):
    
    # lowercase and split by spaces
    words = sentence.lower().split(" ")
    
    return words

In [ ]:
wordfun("The oath of office I take today is an oath of allegiance to all Americans.")

In [ ]:
wordfun("Everyone is listening to you now.")

The way a function works in Python -- when you call the function, it creates a special workspace to carry out the computations. Any variables you make in the "body" of a function lives in this workspace. When your computation is completed, the workspace and all the variables you made will disappear. For example, we creataed "words" in the body of the function, but look at "words" in your main workspace.

In [ ]:
words

It either doesn't exist or isn't the list of words we created. So the body of a function is a special place. We pass data into and out of a function using the argument list in its "**def**"inition, and the "**return**" statement. 

So if you look at the words we are creating, we see something's wrong. The last word is "now." or "americans.", not just "now" and "americans" -- there is a trailing period. So we might want to remove punctuation from the sentence first. We can use the string method replace() for that.

In [ ]:
def wordfun(sentence):
    
    # lowercase and remove punctuation
    clean_sentence = sentence.lower().replace(".","").replace(",","")
    
    # split by spaces
    words = clean_sentence.split(" ")
    
    return words

In [ ]:
wordfun("We will face challenges. We will confront hardships. But we will get the job done.")

Again, you will not find "clean_sentence" in your workspace. It disappeared when Python completed the computation. 

In [ ]:
clean_sentence

As for the task of identifying words in a sentence, our punctuation removal made things better, but it's still not quite right. There's more punctuation to be removed, maybe even quotation marks, and what happens if there are two blank spaces between a word? 

In [ ]:
wordfun("We    will")

So we had better make sure there's only a single space between words. All of this can be fixed up and added to our function, making it more powerful. In the next few weeks, we will see tools from "Natural Language Processing" that do all of this for us, cleanly. 

But for now, let's finish by revisiting the function we made in our last class. I am going to rewrite it a little to make it clearer. I also added more descriptive comments.

In [ ]:
def trend(data,e,n=5):
    
    # subset our summary data to include just the epoch of interest
    hash_subset = data[data["epoch"]==e].copy()

    # sort the hashtags in the epoch by the number of tweets they received
    hashtags = hash_subset.sort_values("tweet count",ascending=False)["hashtag"]
    
    # turn the series into a simple list
    top = list(hashtags[0:n])
    
    # return the top n trends!
    return top

The difference is that I have explicitly added an argument we can use to pass data summarizing the hashtags. 

And let's try it out.

In [ ]:
from pandas import set_option, read_csv, Series
set_option("display.max_colwidth",140)
set_option("display.max_columns",30)
set_option("display.max_rows",50)

In [ ]:
hashsummary = read_csv("http://compute-cuj.org/hashtagsummary.csv")
hashsummary.head(5)

In [ ]:
trend(hashsummary,282,10)

**This is the cleanest way to write a function.** All the data you need for your computation is passed via the argument list and then the results come from the "return" command. Last class, we depended on what are called "scoping rules" to have the function simply find hashsummary. Because we didn't pass it through the argument list, Python searched for an object with that name in our main workspace. It found it and used it. 

There's nothing wrong with that, but it's cleaner to have all your data passed to the function in the arguments and not depend on this mechanism if you don't have to. It will also be cleaner to share this kind of function with people. They can see exactly what they need to do to create a top 10 list, say. They pass epoch number and the length of the list and the data to trend(). Make sense? 

You might choose to summarize the hashtags in a different way, maybe forming a column referring to the average tweets per user. As you change your summary, you can still use the function because each new summary is passed via the argument list. I apologize for the sloppiness.

Here is the trend function using history too. Just in case you need a cleaner version of it. Again, we pass the data (the hash summary we want), the epoch number we're after and the number of topics we want in our list.

In [ ]:
def trendhist(data,epoch,ntopics=5):
    
    # collect the old topics into a list, then turn that list into a series 
    # so we can compute produce value_counts(). this will be the number
    # of times each topic was in the top n list over the last 5 epochs.

    # if you want to change the number of epochs used in the history, 
    # add "nepochs" to the argument list and replace 5 with nepochs below.
    
    oldtopics = []
    
    for e in range(epoch-5,epoch):
        oldtopics = oldtopics + trend(data,e,ntopics)
    
    oldtopics = Series(oldtopics).value_counts()
    
    # now pull the new topics, grabbing extra (3* the number) because we are going 
    # to drop some topics out if they appeared too often in the past. we have to
    # ask for a longer list to make sure we can get "ntopics" worth.
    
    newtopics = trend(data,epoch,3*ntopics)

    # and test if they appeared too many times in the past.
    # here we ask that they appeared in the top 5 less than 3 days,
    # otherwise we leave them out
    
    final = []
    
    print "--"*10
    
    for t in newtopics:
        
        if (not t in oldtopics) or (oldtopics[t] < 3):
            final.append(t)
        else:
            print "Dropping", t
            
    return final[0:ntopics]

In [ ]:
trendhist(hashsummary,290)